## Exercises

Task 0. Build an N-gram language model based on some corpus.

Завдання 0: Побудова N-грам моделі


In [5]:
from collections import defaultdict, Counter
import re

# Вхідний текстовий корпус (невеликий приклад)
corpus = """
    Natural language processing (NLP) is a field of artificial intelligence. 
    NLP helps machines understand and generate human language.
    This field is rapidly evolving with the help of deep learning.
"""

# Оброляємо текст
def preprocess(text):
    text = text.lower()                   # Переводимо у нижній регістр
    text = re.sub(r'[^a-z\s]', '', text)  # Видаляємо розділові знаки
    sentences = text.strip().split("\n")  # Розбиваємо на речення за рядками
    tokenized = [sentence.strip().split() for sentence in sentences if sentence.strip()]  # Токенізуємо
    return tokenized

# Застосовуємо попередню обробку до корпусу
sentences = preprocess(corpus)

# Побудова біграмної моделі (використовуємо defaultdict(Counter) для підрахунку частот)
bigrams = defaultdict(Counter)

# Проходимося по кожному реченню та збираємо пари (слово, наступне слово)
for sentence in sentences:
    for i in range(len(sentence) - 1):
        w1, w2 = sentence[i], sentence[i+1]
        bigrams[w1][w2] += 1  # Збільшуємо лічильник для пари слів

# 4. Виведення результатів
# Виводимо всі знайдені біграми та кількість їх появ у тексті
print("Біграми та їхні частоти:")
for w1 in bigrams:
    for w2 in bigrams[w1]:
        print(f"({w1}, {w2}): {bigrams[w1][w2]}")


Біграми та їхні частоти:
(natural, language): 1
(language, processing): 1
(processing, nlp): 1
(nlp, is): 1
(nlp, helps): 1
(is, a): 1
(is, rapidly): 1
(a, field): 1
(field, of): 1
(field, is): 1
(of, artificial): 1
(of, deep): 1
(artificial, intelligence): 1
(helps, machines): 1
(machines, understand): 1
(understand, and): 1
(and, generate): 1
(generate, human): 1
(human, language): 1
(this, field): 1
(rapidly, evolving): 1
(evolving, with): 1
(with, the): 1
(the, help): 1
(help, of): 1
(deep, learning): 1


Task 1. Compare bi- and tri-gram models

Завдання 1. Порівняйте біграми (2-грамну модель) і триграми (3-грамну модель).

In [7]:
# Побудова триграмної моделі
trigrams = defaultdict(Counter)
for sentence in sentences:
    for i in range(len(sentence) - 2):
        w1, w2, w3 = sentence[i], sentence[i+1], sentence[i+2]
        trigrams[(w1, w2)][w3] += 1

print("\nТриграми та їхні частоти:")
for (w1, w2) in trigrams:
    for w3 in trigrams[(w1, w2)]:
        print(f"({w1}, {w2}, {w3}): {trigrams[(w1, w2)][w3]}")



Триграми та їхні частоти:
(natural, language, processing): 1
(language, processing, nlp): 1
(processing, nlp, is): 1
(nlp, is, a): 1
(is, a, field): 1
(a, field, of): 1
(field, of, artificial): 1
(of, artificial, intelligence): 1
(nlp, helps, machines): 1
(helps, machines, understand): 1
(machines, understand, and): 1
(understand, and, generate): 1
(and, generate, human): 1
(generate, human, language): 1
(this, field, is): 1
(field, is, rapidly): 1
(is, rapidly, evolving): 1
(rapidly, evolving, with): 1
(evolving, with, the): 1
(with, the, help): 1
(the, help, of): 1
(help, of, deep): 1
(of, deep, learning): 1


Task 2. Apply interpolation/backoff to your model so that it can better handle unknown words/prompts.

Завдання 2. Застосуйте інтерполяцію або "відкат" (backoff) до вашої моделі, щоб краще обробляти невідомі слова/підказки.

In [11]:

for sentence in sentences:
    for i in range(len(sentence)):
        unigrams[sentence[i]] += 1
        if i < len(sentence) - 1:
            bigrams[sentence[i]][sentence[i+1]] += 1
        if i < len(sentence) - 2:
            trigrams[(sentence[i], sentence[i+1])][sentence[i+2]] += 1
# 4. Функція з лінійною інтерполяцією
# Використовуємо ваги для комбінації ймовірностей з уніграм, біграм, триграм

def interpolated_probability(w1, w2, w3, l1=0.1, l2=0.3, l3=0.6):
    unigram_prob = unigrams[w3] / sum(unigrams.values()) if w3 in unigrams else 0
    bigram_prob = bigrams[w2][w3] / sum(bigrams[w2].values()) if w2 in bigrams and w3 in bigrams[w2] else 0
    trigram_prob = trigrams[(w1, w2)][w3] / sum(trigrams[(w1, w2)].values()) if (w1, w2) in trigrams and w3 in trigrams[(w1, w2)] else 0
    return l1 * unigram_prob + l2 * bigram_prob + l3 * trigram_prob

# 5. Приклад використання
# Це дозволяє оцінити, яке слово найбільш ймовірне після вказаного контексту
context = ("this", "field")
candidates = ["is", "evolving", "deep", "learning", "unknownword"]

print("\nІнтерпольовані ймовірності для слова після 'this field':")
for word in candidates:
    prob = interpolated_probability(context[0], context[1], word)
    print(f"{word}: {prob:.4f}")



Інтерпольовані ймовірності для слова після 'this field':
is: 0.7569
evolving: 0.0034
deep: 0.0034
learning: 0.0034
unknownword: 0.0000


Task 3. Use this model to build sentences. Meaning, for a prompt consisting of words p1,...,pn, it should produce a continuation w1,...,wk.

Завдання 3. Використовуйте цю модель для побудови речень. Тобто, для підказки, що складається зі слів p1,...,pn, модель повинна передбачити продовження wn,…,wk.

In [15]:
# Генерація речення з підказки (prompt)
def generate_sentence(prompt, max_words=10):
    generated = prompt[:]
    for _ in range(max_words):
        if len(generated) < 2:
            break
        w1, w2 = generated[-2], generated[-1]
        candidates = list(unigrams.keys())
        # Знаходимо слово з найбільшою ймовірністю
        next_word = max(candidates, key=lambda w: interpolated_probability(w1, w2, w))
        generated.append(next_word)
    return " ".join(generated)

# 6. Приклад побудови речення
prompt = ["this", "field"]
generated_sentence = generate_sentence(prompt)
print("\nЗгенероване речення:")
print(generated_sentence)


Згенероване речення:
this field is rapidly evolving with the help of deep learning language
